### Importing Necessary Libraries.

In [1]:
import pygame
import os
import random
import numpy

pygame 2.1.2 (SDL 2.0.18, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Defining the Board, Block Size, Population and Mating Pool size.

In [2]:
M=10
N=10
grid_size=25
population_length=500         
parents_length=50
file_name= 'Snake Game Using Genetic Algorithm.npz'

### Function to randomly generate food in the grid.

In [3]:
def food():
    global Food
    snake_no_grids= [i for i in grids if i not in Snake]
    Food = random.choice(snake_no_grids)

### Function to Update Snake based on the move given by the Neural Network.

In [4]:
def update_snake():
    global snake_tail,snake_head,snake_body
    display()
    pygame.time.wait(pause_time)
    (x,y)=Snake[0]
    Snake.insert(0,(x+1,y)) if action == 'Right' else Snake.insert(0,(x-1,y)) if action == 'Left' else Snake.insert(0,(x,y+1)) if action == 'Bottom' else Snake.insert(0,(x,y-1))
    snake_tail=Snake.pop()
    snake_head,snake_body=Snake[0],Snake[1:]

### Function to define the Snake Game
#### This function runs for each individual and for every generation.

In [5]:
def Snake_game():
    global Snake,screen,loop,mloop,pause_time
    
    pygame.init()
    screen = pygame.display.set_mode((M*grid_size,N*grid_size))
    
    (x1,y1)=random.choice(grids)
    (x2,y2)=random.choice([(x1,y1+1),(x1-1,y1),(x1,y1-1),(x1+1,y1)]) # These randomly generate snake's starting position in the gird.
    
    Snake,steps,uniq,pause_time=[(x1,y1),(x2,y2)],0,[0]*(M*N-2),0
    food()
    
    loop=True                                                        # The below loop runs until the Snake Dies.
    while loop:
        steps=steps+1
    
        prediction_from_genetic_weights()
        
        update_snake()                                              # This function Updates the snakes length based on the predictions made by the neural network.
        
        if len(Snake)==M*N:
            print('Hurray! Snake achieved the maximum score')       # If the length of the Snake is 100. Game ends.
            loop=False
        
        elif snake_head==Food:                                      # If snake consumes food, increase the tail length and randomly generate food at other position.
            food()
            Snake.append(snake_tail)
            pause_time =key_sensitive*5  if len(Snake)==M*N-10 else pause_time
        
        elif snake_head not in grids or snake_head in snake_body:  # If snake head has hit the wall or its own body, end the game.
            loop=False
        
        
        ev=pygame.event.get()
        for event in ev:
            if event.type == pygame.QUIT:                          # If the user wants to quit, end the program.
                pygame.quit()
                mloop,loop=False,False
            elif event.type == pygame.KEYDOWN:
                pause_time = pause_time+key_sensitive  if event.key == pygame.K_UP else pause_time-key_sensitive if event.key == pygame.K_DOWN and pause_time>=key_sensitive else pause_time
        
        if (Snake[0],Food) not in uniq:
            uniq.append((Snake[0],Food))
            del uniq[0]
        
        else:
            loop=False
    score=len(Snake)-2                                            # Getting the Score of the Snake when the game ends.
    
    return (score+0.5+0.5*(score-steps/(score+1))/(score+steps/(score+1)))*1000000,score,steps  # Return the Fitness value of the current snake.

### Function to predict the move of the individual Snake based on the Snake's position and pass the input to neural network

In [6]:
def prediction_from_genetic_weights():
    global action
    lstop=[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]                           # To get the head direction to include in the input.
    
    lst=[(0,-1),(1,0),(0,1),(-1,0)]
    
    head_diriction=lstop[lst.index((Snake[0][0]-Snake[1][0],Snake[0][1]-Snake[1][1]))] # Get head direction based on the Snake head and its next block.
    (x,y)=Snake[0]                                                           # Snake Head.
    # Observe in all the 8 directions.
    
    d1=[(x,_) for _ in range(y-1,-1,-1)]
    
    d3=[(_,y) for _ in range(x+1,M)]
    
    d5=[(x,_) for _ in range(y+1,N)]
    
    d7=[(_,y) for _ in range(x-1,-1,-1)]
    
    d8=[(x-_,y-_) for _ in range(1,min(x,y)+1)]
    
    d4=[(x+_,y+_) for _ in range(1,min(M-x,N-y))]
    
    d2=[(x+_,y-_) for _ in range(1,max(M-x,N-y)) if (x+_,y-_) in grids]
    
    d6=[(x-_,y+_) for _ in range(1,max(M-x,N-y))  if (x-_,y+_) in grids]
    
    d,val=[d1,d2,d3,d4,d5,d6,d7,d8],min(M,N)-1
    
    wall_distance=[len(i)/val for i in d]                                 # Calculate the distance of the wall in each direction.
    
    food_presence=[(val-j.index(Food))/val if Food in j else 0 for j in d]# Calculate Food Presence in each direction.
    
    body_presence=[min([dv.index(v) if v in Snake else val for v in dv])/val if dv else 0 for dv in d] # Calculate Body Presence in each direction.
    
    vision=[j[i] for i in range(8) for j in [wall_distance,body_presence,food_presence]] # Combine 3 attributes for every direction.
    
    input_layer=vision+head_diriction                                    # Combine 24 attributes with 4 head direction attributes to pass to neural network.
    
    action=neural_network(input_layer)

### Neural Network and Activation Functions.

In [7]:
def neural_network(ip):
    m1,s1=numpy.reshape(ip,(1,NN[0])),0
    for _ in range(len(AF)):
        l1,l2=NN[_],NN[_+1]
        s2,s3=s1+l1*l2,s1+l2+l1*l2
        m2,m3=numpy.reshape(weights[s1:s2],(l1,l2)),numpy.reshape(weights[s2:s3],(1,l2))
        m4=numpy.matmul(m1,m2)+m3
        m1,s1=AF[_](m4),s3
    return Actions[numpy.argmax(m1)]

def relu(x):
    lst=[_ if _>0 else 0 for _ in x[0]]
    return numpy.array(lst)

def sigmoid(x):
    lst=[1/(1+(numpy.exp(-1*_))) for _ in x[0]]
    return numpy.array(lst)

### Function to display PyGame window.

In [8]:
def display():
    pygame.draw.rect(screen,(0,0,0), (0,0,M*grid_size,N*grid_size))
    pygame.draw.rect(screen,(255,255,255),(Snake[0][0]*grid_size,Snake[0][1]*grid_size,grid_size,grid_size))
    [pygame.draw.rect(screen,(255,255,255), (i[0]*grid_size,i[1]*grid_size,grid_size,grid_size),1) for i in Snake[1:]]
    pygame.draw.rect(screen,(0,255,0), (Food[0]*grid_size,Food[1]*grid_size,grid_size,grid_size))
    pygame.display.update()

### Function to perform Random CrossOver using Roulette Wheel Selection.

In [9]:
def crossover():
    global offspring
    offspring=[]
    for _ in range(population_length-parents_length):
        parent1_id=random.choice(Roulette_wheel)
        parent2_id=random.choice(Roulette_wheel)
        while parent2_id==parent1_id: parent2_id=random.choice(Roulette_wheel)
        wts=[parents[parent1_id][i] if random.uniform(0, 1) < 0.5 else parents[parent2_id][i] for i in range(weights_length)]
        offspring.append(wts)

### Function to perform Mutation on OffSprings.

In [10]:
def mutation():
    global offspring
    for i in range(population_length-parents_length):
        for _ in range(int(weights_length*0.05)):
            plc=random.randint(0,weights_length-1)
            value=random.choice(numpy.arange(-0.5,0.5,step=0.001))
            offspring[i][plc]=offspring[i][plc]+value

### Defining Parameters of NN, GA and Snake Game.

In [11]:
NN=[28,8,4]
AF=[relu,sigmoid]
pause_time,key_sensitive,generation_length,mloop=0,15,250,True
grids=[(i,j) for i in range(M) for j in range(N)]
Actions=['Top','Right','Bottom','Left']
Roulette_wheel=list(range(0,int(0.2*parents_length)))*3+list(range(int(0.2*parents_length),int(0.5*parents_length)))*2+list(range(int(0.5*parents_length),parents_length))
weights_length=sum([NN[_]*NN[_+1]+NN[_+1] for _ in range(len(NN)-1)])

In [12]:
if file_name not in os.listdir(os.getcwd()):
    population,statis=numpy.random.choice(numpy.arange(-1,1,step=0.001),size=(population_length,weights_length),replace=True),numpy.array([0,0,0,0])
    Generation,High_score=1,0
else:
    IP=numpy.load(file_name)
    population,statis=IP['POPULATION'],IP['STATIS']
    Generation,High_score=statis[-1][0]+1,statis[-1][-1]

### Run the Snake game for n Generations.

In [13]:
Generation = 0
while Generation<=generation_length and mloop:
    # Each Generation.
    print('###################### ','Generation ',Generation,' ######################')
    Fitness,Score,i=[],[],0
    while i<population_length and mloop:
        # For Every Individual in the Population.
        
        weights,i=list(population[i,:]),i+1
        
        fittness,score,steps=Snake_game()            # Calculate the Fitness, Score, Steps a  individual takes.
#         print('Chromosome ',"{:03d}".format(i),' >>> ','Score : ',"{:03d}".format(score),', Steps : ',"{:04d}".format(steps),', Fittness : ',fittness)
        
        Fitness.append(fittness),Score.append(score) # Append fitness, score of each individual for a generation.
    
    parents,max_fittness,avg_score,j=[],max(Fitness),sum(Score)/len(Score),0  # Get the top 50 individuals with Max Fitness to be the Mating Pool.
    
    while j<parents_length and mloop:
        j,parent_id=j+1,Fitness.index(max(Fitness))
        Fitness[parent_id]=-999
        parents.append(list(population[parent_id,:]))  # Append all the parents to a list.
        
    # After getting parents from a single generation    
    while mloop and j==parents_length:
        j=j+1
        
        High_score=max(Score) if max(Score) > High_score else High_score
        
        print('Generation high score : ',max(Score))
        
        crossover() # Perform CrossOver operation on parents.
        mutation()  # Perform Mutation for OffSprings.
        
        statis=numpy.row_stack((statis,numpy.array([Generation,max(Score),avg_score,High_score])))
        
        population=numpy.reshape(parents+offspring,(population_length,-1))  # Fill the Population with Parents and the OffSprings produced.
    Generation=Generation+1
    
pygame.quit()  # Quit the Game if any of the Stopping Criteria is met.
numpy.savez(file_name,POPULATION=population,STATIS=statis)  # Save the Final Population to a file.

######################  Generation  0  ######################
Generation high score :  1
######################  Generation  1  ######################
Generation high score :  2
######################  Generation  2  ######################
Generation high score :  2
######################  Generation  3  ######################
Generation high score :  2
######################  Generation  4  ######################
Generation high score :  1
######################  Generation  5  ######################
Generation high score :  2
######################  Generation  6  ######################
Generation high score :  2
######################  Generation  7  ######################
Generation high score :  1
######################  Generation  8  ######################
Generation high score :  1
######################  Generation  9  ######################
Generation high score :  2
######################  Generation  10  ######################
Generation high score :  2
####################

Generation high score :  30
######################  Generation  92  ######################
Generation high score :  30
######################  Generation  93  ######################
Generation high score :  33
######################  Generation  94  ######################
Generation high score :  37
######################  Generation  95  ######################
Generation high score :  31
######################  Generation  96  ######################
Generation high score :  33
######################  Generation  97  ######################
Generation high score :  34
######################  Generation  98  ######################
Generation high score :  33
######################  Generation  99  ######################
Generation high score :  34
######################  Generation  100  ######################
Generation high score :  33
######################  Generation  101  ######################
Generation high score :  34
######################  Generation  102  ###################

Generation high score :  45
######################  Generation  182  ######################
Generation high score :  44
######################  Generation  183  ######################
Generation high score :  41
######################  Generation  184  ######################
Generation high score :  46
######################  Generation  185  ######################
Generation high score :  45
######################  Generation  186  ######################
Generation high score :  50
######################  Generation  187  ######################
Generation high score :  47
######################  Generation  188  ######################
Generation high score :  46
######################  Generation  189  ######################
Generation high score :  47
######################  Generation  190  ######################
Generation high score :  46
######################  Generation  191  ######################
Generation high score :  45
######################  Generation  192  ###########